In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#Currently, my code only has capability to do frame-by-frame analysis on 1 video.  Perhaps future functionality will
#be able to expand beyond this.  

name = 'confocal_1000ms_60x'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.83, 1.02, 1)
conversion[2] = (0.83, 1.02, 1)
conversion[3] = (0.83, 1.02, 1)
# conversion[4] = (0.30, 5.31, 1)
# conversion[5] = (0.30, 5.33, 1)
# conversion[6] = (0.30, 5.33, 1)
# conversion[7] = (0.30, 5.35, 1)

trajectory = dict()
cut = 5

trajectory[1] = np.genfromtxt('./confocal_1000ms_60x_1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('./confocal_1000ms_60x_2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('./confocal_1000ms_60x_3.csv',
            delimiter =",")
# trajectory[4] = np.genfromtxt('./CON_40x_2D_130ms_4_T.csv',
#             delimiter =",")
# trajectory[5] = np.genfromtxt('./CON_40x_2D_130ms_5_T.csv',
#             delimiter =",")
# trajectory[6] = np.genfromtxt('./CON_40x_2D_130ms_6_T.csv',
#             delimiter =",")
# trajectory[7] = np.genfromtxt('./CON_40x_2D_130ms_7_T.csv',
#             delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Collect trajectories of separate videos into one array
#Comment this section out to make useable for a single trajectory
parts = dict()
placeholder = trajectory[1]
tots = dict()

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1

#     while counter < parts[num]:
#         counter = counter + 1
#         placeholder = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    trajectory[num][:,0] = trajectory[num][:,0] + tots[num-1]
    tots[num] = tots[num] + tots[num-1]
    placeholder = np.append(placeholder, trajectory[num], axis=0)

trajectory[1] = placeholder

# trajectory[2][:,0] = trajectory[2][:,0] + tots[1]
# placeholder = np.append(placeholder, trajectory[2], axis=0)
# trajectory[3][:,0] = trajectory[3][:,0] + tots[2]
# placeholder = np.append(placeholder, trajectory[3], axis=0)

In [3]:
#Theoretical diffusion calculation using Stokes-Einstein

k = 1.380649 * 10**-23
TC = 28
TK = TC + 273
mu1 = 1.002*10**-3; mu2 = 0.798*10**-3; TC1 = 20; TC2 = 30

mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1
radius = (128/2) * 10**-9 #m
Deff1 = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2 = Deff1 * 10**12 #um2/s
Deff2

4.106863414779643

In [4]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    """
    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [5]:
rawframes = trajectory[1][:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

x = dict()
y = dict()
z = dict()
M1x = dict()
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

In [6]:
dataset = dict()

particles = trajectory[1][:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = trajectory[1][:, :]

#Perform conversion on raw dataset
rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (rawdataset[min1:max1+1, 0:5])

In [7]:
#I is a set of all nonzero elements in trajectories.
I = dict()
for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]

In [8]:
np.count_nonzero(x[300])

46

In [9]:
total1

801

In [10]:
#I wrote a short bit of code to filter out trajectories that are too short to be included.  This also reduces computing
#time by cutting down on MSDs that need to be calculated.  
cut = 5
cutoff = cut
x1 = dict()
y1 = dict()
z1 = dict()
fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        z1[num - nones] = z[num]
        # I must also redefine the particle numbers to reflect the new set.
        #teancum[num - nones][:, 0] = fifties

x = x1
y = y1
z = z1

In [11]:
#fifties is the new total1 after filtering out short trajectories
total1 = fifties + 1
fifties

621

In [12]:
for num in range(1, total1):

    #Construct MSD and Deff dictionaries
    M1x[num] = np.zeros(frames.shape[0])
    M1x[num][0:frames.shape[0]] = 0
    M1y[num] = np.zeros(frames.shape[0])
    M1y[num][0:frames.shape[0]] = 0
    M1z[num] = np.zeros(frames.shape[0])
    M1z[num][0:frames.shape[0]] = 0
    M2xy[num] = np.zeros(frames.shape[0])
    M2xy[num][0:frames.shape[0]] = 0
    M2xz[num] = np.zeros(frames.shape[0])
    M2xz[num][0:frames.shape[0]] = 0
    M2yz[num] = np.zeros(frames.shape[0])
    M2yz[num][0:frames.shape[0]] = 0
    M3[num] = np.zeros(frames.shape[0])
    M3[num][0:frames.shape[0]] = 0
    
    #Fill in MSD and Deff dictionaries
    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    startz = z[num][first]
    
    for num1 in range(first, last):
        M1x[num][num1] = (x[num][num1] - startx)**2
        M1y[num][num1] = (y[num][num1] - starty)**2
        M1z[num][num1] = (z[num][num1] - startz)**2
        M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2
        M2xz[num][num1] = (x[num][num1] - startx)**2 + (z[num][num1] - startz)**2
        M2yz[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2
        M3[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2 + (x[num][num1] - startx)**2
        
#         M1x[num][first] = 'NaN'
#         M1y[num][first] = 'NaN'
#         M1z[num][first] = 'NaN'
#         M2xy[num][first] = 'NaN'
#         M2xz[num][first] = 'NaN'
#         M2yz[num][first] = 'NaN'
#         M3[num][first] = 'NaN'
    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M1z[num] = ma.masked_equal(M1z[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    M2xz[num] = ma.masked_equal(M2xz[num], 0)
    M2yz[num] = ma.masked_equal(M2yz[num], 0)
    M3[num] = ma.masked_equal(M3[num], 0)

In [13]:
#np.count_nonzero(M1x[495])
shift = next((i for i, x in enumerate(M1x[50]) if x), None)
sM1x = dict()
sM1x[50] = np.zeros(frames.shape[0] - shift)
for num in range(0, M1x[50].shape[0] - shift):
    sM1x[50][num] = M1x[50][num + shift]
sM1x

{50: array([  4.40896000e-05,   1.16424100e-04,   1.99092100e-04,
          2.23823610e-03,   5.58009000e-05,   1.15804090e-03,
          7.05433600e-04,   2.75560000e-06,   8.43902500e-04,
          6.62032900e-04,   1.16424100e-04,   2.75560000e-06,
          2.16039040e-03,   4.30562500e-04,   1.72225000e-03,
          2.23823610e-03,   2.64813160e-03,   2.75560000e-06,
          1.10224000e-03,   1.86278560e-03,   4.65696400e-04,
          1.04781690e-03,   1.16424100e-04,   4.51987290e-03,
          3.67114810e-03,   3.77241640e-03,   4.40896000e-05,
          1.10224000e-03,   3.77241640e-03,   2.16039040e-03,
          7.50212100e-04,   4.74583210e-03,   2.48004000e-03,
          2.56339690e-03,   1.04781690e-03,   2.16039040e-03,
          1.86278560e-03,   4.30562500e-04,   5.21428410e-03,
          1.27377610e-03,   2.91060250e-03,   2.82173440e-03,
          9.43104100e-04,   3.09247210e-03,   7.74048040e-03])}

In [14]:
M1x[50]

masked_array(data = [-- 4.408959999996434e-05 0.00011642410000000167 0.00019909210000006454
 0.002238236099999618 5.5800899999968134e-05 0.001158040900000091
 0.0007054336000001843 2.755600000003669e-06 0.000843902499999885
 0.0006620329000001502 0.00011642410000000167 2.755600000003669e-06
 0.002160390400000234 0.0004305624999999835 0.001722249999999934
 0.002238236099999618 0.002648131599999869 2.755600000003669e-06
 0.001102240000000052 0.0018627856000000267 0.0004656964000000067
 0.001047816900000015 0.00011642410000000167 0.0045198729000002855
 0.0036711480999997216 0.0037724163999997857 4.408959999996434e-05
 0.001102240000000052 0.0037724163999997857 0.002160390400000234
 0.0007502120999998311 0.004745832100000445 0.0024800399999997634
 0.002563396900000535 0.001047816900000015 0.002160390400000234
 0.0018627856000000267 0.0004305624999999835 0.005214284099999759
 0.0012737761000001743 0.002910602500000042 0.0028217343999999826
 0.0009431040999999464 0.003092472100000166 0.00774

In [15]:
#Each entry in yes is an array of particles that are in the given frame.  The frame is given by the index of yes.
yes = dict()
yessize = dict()
for num1 in range(1, frames.shape[0]):
    count = 0
    yes[num1] = np.zeros(total1).astype(np.int64)
    for num in range(1, total1):
        check = num1 in I[num]
        if check == True:
            yes[num1][count] = num
            count = count + 1
    iindex = np.where(yes[num1]==max(yes[num1]))[0][0]
    yes[num1] = yes[num1][0:iindex]

In [16]:
#And the FBF analyis
framed = dict()
fM1y = dict()
fM1z = dict()
fM2xy = dict()
fM2xz = dict()
fM2yz = dict()
fM3 = dict()

for num2 in range(1, frames.shape[0]):
    framed[num2] = np.zeros(frames.shape[0])
    fM1y[num2] = np.zeros(frames.shape[0])
    fM1z[num2] = np.zeros(frames.shape[0])
    fM2xy[num2] = np.zeros(frames.shape[0])
    fM2xz[num2] = np.zeros(frames.shape[0])
    fM2yz[num2] = np.zeros(frames.shape[0])
    fM3[num2] = np.zeros(frames.shape[0])
    
    for num1 in range(0, frames.shape[0]):
        frame = np.zeros(yes[num2].shape[0])
        fmM1y = np.zeros(yes[num2].shape[0])
        fmM1z = np.zeros(yes[num2].shape[0])
        fmM2xy = np.zeros(yes[num2].shape[0])
        fmM2xz = np.zeros(yes[num2].shape[0])
        fmM2yz = np.zeros(yes[num2].shape[0])
        fmM3 = np.zeros(yes[num2].shape[0])
        
        for num in range(0, yes[num2].shape[0]):
            frame[num] = M1x[yes[num2][num]][num1]
            fmM1y[num] = M1y[yes[num2][num]][num1]
            fmM1z[num] = M1z[yes[num2][num]][num1]
            fmM2xy[num] = M2xy[yes[num2][num]][num1]
            fmM2xz[num] = M2xz[yes[num2][num]][num1]
            fmM2yz[num] = M2yz[yes[num2][num]][num1]
            fmM3[num] = M3[yes[num2][num]][num1]
        frame = ma.masked_invalid(frame)
        fmM1y = ma.masked_invalid(fmM1y)
        fmM1z = ma.masked_invalid(fmM1z)
        fmM2xy = ma.masked_invalid(fmM2xy)
        fmM2xz = ma.masked_invalid(fmM2xz)
        fmM2yz = ma.masked_invalid(fmM2yz)
        fmM3 = ma.masked_invalid(fmM3)
        
        framed[num2][num1] = stat.gmean(frame)
        fM1y[num2][num1] = stat.gmean(fmM1y)
        fM1z[num2][num1] = stat.gmean(fmM1z)
        fM2xy[num2][num1] = stat.gmean(fmM2xy)
        fM2xz[num2][num1] = stat.gmean(fmM2xz)
        fM2yz[num2][num1] = stat.gmean(fmM2yz)
        fM3[num2][num1] = stat.gmean(fmM3)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [17]:
FM1x = np.zeros(frames.shape[0])
FM1y = np.zeros(frames.shape[0])
FM1z = np.zeros(frames.shape[0])
FM2xy = np.zeros(frames.shape[0])
FM2xz = np.zeros(frames.shape[0])
FM2yz = np.zeros(frames.shape[0])
FM3 = np.zeros(frames.shape[0])

for num in range(1, frames.shape[0]):
    FM1x = FM1x + framed[num]
    FM1y = FM1y + fM1y[num]
    FM1z = FM1z + fM1z[num]
    FM2xy = FM2xy + fM2xy[num]
    FM2xz = FM2xz + fM2xz[num]
    FM2yz = FM2yz + fM2yz[num]
    FM3 = FM3 + fM3[num]
    
FM1x = FM1x/frames.shape[0]
FM1y = FM1y/frames.shape[0]
FM1z = FM1z/frames.shape[0]
FM2xy = FM2xy/frames.shape[0]
FM2xz = FM2xz/frames.shape[0]
FM2yz = FM2yz/frames.shape[0]
FM3 = FM3/frames.shape[0]



In [18]:
FD1x = FM1x/time
FD1y = FM1y/time
FD1z = FM1z/time
FD2xy = FM2xy/time
FD2xz = FM2xz/time
FD2yz = FM2yz/time
FD3 = FM3/time

In [19]:
def plot_MLOG(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
    #I will need to calculate standard deviations eventually
#     #Now to calculate the standard dev at each point:
#     for num in range (1, total1):
#         SDunit = (MSD[num] - MMSD)**2
#         SD = SD + SDunit
#     SD = np.sqrt(SD/total1)
#     SE = SD/np.sqrt(total1)
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    
#     ax.plot(t, linel[0], linewidth=3, color='blue')
#     ax.plot(t, linel[1], linewidth=3, color='red')
#     ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([0.1, limit1])
    plt.gca().set_ylim([0.0001, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [20]:
one1, bleh = plot_MLOG(FM2xy, FM1x, FM1y, time, 1, 1, 'MSD (um^2)', logplot, 100.1, 0.1, 1, 1)
plt.show()

In [21]:
def plot_M(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [22]:
one1, bleh = plot_M(FM2xy, FM1x, FM1y, time, 0, 0, 'MSD (um^2)', Mplot, 50.1, 0.0101, 10, 0.002)
plt.show()

In [23]:
Deff3 = Deff2*np.ones(time.shape[0])

In [24]:
def plot_M1(M2xy, Mx, My, Mth, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linewidth=10, label='2D', color='blue')
    ax.plot(t, Mx, linewidth=10, label='1D x', color='red')
    ax.plot(t, My, linewidth=10, label='1D x', color='green')
    ax.plot(t, Mth, '--', linewidth=10, label='theoretical', color='purple')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)

    hfont = {'fontname':'Arial'}
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'D ($\mu$m$^2$/s)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.10, 0.54), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [25]:
one1, bleh = plot_M1(FD2xy, FD1x, FD1y, Deff3, time, 0, 1, 'MSD (um^2)', Dplot, 50.1, 0.0021, 10, 0.0004)
plt.show()

In [26]:
#This block will create dictionaries of shifted trajectories to calculate arithmetic and geometric means.  I need to so
#I can compare FBF analysis with them.

final, tots = prettify(trajectory[1], 20, 20, conversion[1][0], conversion[1][1], conversion[1][2])

SM1x = dict()
SM1y = dict()
SM1z = dict()
SM2xy = dict()
SM2xz = dict()
SM2yz = dict()
SM3 = dict()

SD1x = dict()
SD1y = dict()
SD1z = dict()
SD2xy = dict()
SD2xz = dict()
SD2yz = dict()
SD3 = dict()

time2 = final[2][:, 15]

for num in range(1, tots + 1):
    SM1x[num] = final[num][:, 12]
    SM1y[num] = final[num][:, 13]
    SM1z[num] = final[num][:, 14]
    SM2xy[num] = final[num][:, 9]
    SM2xz[num] = final[num][:, 10]
    SM2yz[num] = final[num][:, 11]
    SM3[num] = final[num][:, 8]

    SD1x[num] = final[num][:, 20]
    SD1y[num] = final[num][:, 21]
    SD1z[num] = final[num][:, 22]
    SD2xy[num] = final[num][:, 17]
    SD2xz[num] = final[num][:, 18]
    SD2yz[num] = final[num][:, 19]
    SD3[num] = final[num][:, 16]
    
    SM1x[num] = ma.masked_equal(SM1x[num], 0)
    SM1y[num] = ma.masked_equal(SM1y[num], 0)
    SM1z[num] = ma.masked_equal(SM1z[num], 0)
    SM2xy[num] = ma.masked_equal(SM2xy[num], 0)
    SM2xz[num] = ma.masked_equal(SM2xz[num], 0)
    SM2yz[num] = ma.masked_equal(SM2yz[num], 0)
    SM3[num] = ma.masked_equal(SM3[num], 0)
    
    SD1x[num] = ma.masked_equal(SD1x[num], 0)
    SD1y[num] = ma.masked_equal(SD1y[num], 0)
    SD1z[num] = ma.masked_equal(SD1z[num], 0)
    SD2xy[num] = ma.masked_equal(SD2xy[num], 0)
    SD2xz[num] = ma.masked_equal(SD2xz[num], 0)
    SD2yz[num] = ma.masked_equal(SD2yz[num], 0)
    SD3[num] = ma.masked_equal(SD3[num], 0)

In [27]:
tots

491

In [28]:
trajectory[1][:,0]

array([   1.,    1.,    1., ...,  799.,  800.,  800.])

In [29]:
#Now to calculate arithmetic and geometric means.

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM1z = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])
arM2xz = np.zeros(SM1x[1].shape[0])
arM2yz = np.zeros(SM1x[1].shape[0])
arM3 = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD1z = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])
arD2xz = np.zeros(SM1x[1].shape[0])
arD2yz = np.zeros(SM1x[1].shape[0])
arD3 = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM1z = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])
st_arM2xz = np.zeros(SM1x[1].shape[0])
st_arM2yz = np.zeros(SM1x[1].shape[0])
st_arM3 = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD1z = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])
st_arD2xz = np.zeros(SM1x[1].shape[0])
st_arD2yz = np.zeros(SM1x[1].shape[0])
st_arD3 = np.zeros(SM1x[1].shape[0])

# for num in range(1, tots +1):
#     arM1x = arM1x + SM1x[num]
#     arM1y = arM1y + SM1y[num]
#     arM1z = arM1z + SM1z[num]
#     arM2xy = arM2xy + SM2xy[num]
#     arM2xz = arM2xz + SM2xz[num]
#     arM2yz = arM2yz + SM2yz[num]
#     arM3 = arM3 + SM3[num]
    
#     arD1x = arD1x + SD1x[num]
#     arD1y = arD1y + SD1y[num]
#     arD1z = arD1z + SD1z[num]
#     arD2xy = arD2xy + SD2xy[num]
#     arD2xz = arD2xz + SD2xz[num]
#     arD2yz = arD2yz + SD2yz[num]
#     arD3 = arD3 + SD3[num]
    
#     arM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
#     gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
#     gM1z[num2+1] = ma.masked_invalid(gM1z[num2+1])
#     gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])
#     gM2xz[num2+1] = ma.masked_invalid(gM2xz[num2+1])
#     gM2yz[num2+1] = ma.masked_invalid(gM2yz[num2+1])
#     gM3[num2+1] = ma.masked_invalid(gM3[num2+1])
        
#     gD1x[num2+1] = ma.masked_invalid(SD1x[num2+1])
#     gD1y[num2+1] = ma.masked_invalid(SD1y[num2+1])
#     gD1z[num2+1] = ma.masked_invalid(SD1z[num2+1])
#     gD2xy[num2+1] = ma.masked_invalid(SD2xy[num2+1])
#     gD2xz[num2+1] = ma.masked_invalid(SD2xz[num2+1])
#     gD2yz[num2+1] = ma.masked_invalid(SD2yz[num2+1])
#     gD3[num2+1] = ma.masked_invalid(SD3[num2+1])
    
# arM1x = arM1x/tots
# arM1y = arM1y/tots
# arM1z = arM1z/tots
# arM2xy = arM2xy/tots
# arM2xz = arM2xz/tots
# arM2yz = arM2yz/tots
# arM3 = arM3/tots

# arD1x = arD1x/tots
# arD1y = arD1y/tots
# arD1z = arD1z/tots
# arD2xy = arD2xy/tots
# arD2xz = arD2xz/tots
# arD2yz = arD2yz/tots
# arD3 = arD3/tots

In [30]:
gM1x = dict()
gM1y = dict()
gM1z = dict()
gM2xy = dict()
gM2xz = dict()
gM2yz = dict()
gM3 = dict()

# log_gM1x = dict()
# log_gM1y = dict()
# log_gM1z = dict()
# log_gM2xy = dict()
# log_gM2xz = dict()
# log_gM2yz = dict()
# log_gM3 = dict()

# gD1x = dict()
# gD1y = dict()
# gD1z = dict()
# gD2xy = dict()
# gD2xz = dict()
# gD2yz = dict()
# gD3 = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM1z = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])
geoM2xz = np.zeros(SM1x[1].shape[0])
geoM2yz = np.zeros(SM1x[1].shape[0])
geoM3 = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM1z = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])
st_geoM2xz = np.zeros(SM1x[1].shape[0])
st_geoM2yz = np.zeros(SM1x[1].shape[0])
st_geoM3 = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD1z = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])
geoD2xz = np.zeros(SM1x[1].shape[0])
geoD2yz = np.zeros(SM1x[1].shape[0])
geoD3 = np.zeros(SM1x[1].shape[0])

st_geoD1x = np.zeros(SM1x[1].shape[0])
st_geoD1y = np.zeros(SM1x[1].shape[0])
st_geoD1z = np.zeros(SM1x[1].shape[0])
st_geoD2xy = np.zeros(SM1x[1].shape[0])
st_geoD2xz = np.zeros(SM1x[1].shape[0])
st_geoD2yz = np.zeros(SM1x[1].shape[0])
st_geoD3 = np.zeros(SM1x[1].shape[0])

for num2 in range(0, SM1x[1].shape[0]-1):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM1z[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)
    gM2xz[num2+1] = np.zeros(tots)
    gM2yz[num2+1] = np.zeros(tots)
    gM3[num2+1] = np.zeros(tots)
    
#     gD1x[num2+1] = np.zeros(tots)
#     gD1y[num2+1] = np.zeros(tots)
#     gD1z[num2+1] = np.zeros(tots)
#     gD2xy[num2+1] = np.zeros(tots)
#     gD2xz[num2+1] = np.zeros(tots)
#     gD2yz[num2+1] = np.zeros(tots)
#     gD3[num2+1] = np.zeros(tots)
    
    for num in range(1, tots + 1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM1z[num2+1][num-1] = SM1z[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]
        gM2xz[num2+1][num-1] = SM2xz[num][num2]
        gM2yz[num2+1][num-1] = SM2yz[num][num2]
        gM3[num2+1][num-1] = SM3[num][num2]
        
#         gD1x[num2+1][num-1] = SD1x[num][num2]
#         gD1y[num2+1][num-1] = SD1y[num][num2]
#         gD1z[num2+1][num-1] = SD1z[num][num2]
#         gD2xy[num2+1][num-1] = SD2xy[num][num2]
#         gD2xz[num2+1][num-1] = SD2xz[num][num2]
#         gD2yz[num2+1][num-1] = SD2yz[num][num2]
#         gD3[num2+1][num-1] = SD3[num][num2]
        
    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM1z[num2+1] = ma.masked_invalid(gM1z[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])
    gM2xz[num2+1] = ma.masked_invalid(gM2xz[num2+1])
    gM2yz[num2+1] = ma.masked_invalid(gM2yz[num2+1])
    gM3[num2+1] = ma.masked_invalid(gM3[num2+1])

    #I need to include this, because geometric means can't be calculated on values of 0.
    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM1z[num2+1] = ma.masked_equal(gM1z[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)
    gM2xz[num2+1] = ma.masked_equal(gM2xz[num2+1], 0)
    gM2yz[num2+1] = ma.masked_equal(gM2yz[num2+1], 0)
    gM3[num2+1] = ma.masked_equal(gM3[num2+1], 0)

#     log_gM1x[num2+1] = np.log(gM1x[num2+1])
#     log_gM1y[num2+1] = np.log(gM1y[num2+1])
#     log_gM1z[num2+1] = np.log(gM1z[num2+1])
#     log_gM2xy[num2+1] = np.log(gM2xy[num2+1])
#     log_gM2xz[num2+1] = np.log(gM2xz[num2+1])
#     log_gM2yz[num2+1] = np.log(gM2yz[num2+1])
#     log_gM3[num2+1] = np.log(gM3[num2+1])
        
#     gD1x[num2+1] = ma.masked_invalid(SD1x[num2+1])
#     gD1y[num2+1] = ma.masked_invalid(SD1y[num2+1])
#     gD1z[num2+1] = ma.masked_invalid(SD1z[num2+1])
#     gD2xy[num2+1] = ma.masked_invalid(SD2xy[num2+1])
#     gD2xz[num2+1] = ma.masked_invalid(SD2xz[num2+1])
#     gD2yz[num2+1] = ma.masked_invalid(SD2yz[num2+1])
#     gD3[num2+1] = ma.masked_invalid(SD3[num2+1])
        
    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM1z[num2] = stat.gmean(gM1z[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])
    geoM2xz[num2] = stat.gmean(gM2xz[num2+1])
    geoM2yz[num2] = stat.gmean(gM2yz[num2+1])
    geoM3[num2] = stat.gmean(gM3[num2+1])
    
    #The geometric standard deviation is a function of the mean of the logarithms.
#     st_geoM1x[num2+1] = np.exp(np.sqrt(np.mean((log_gM1x[num2+1] - np.log(geoM1x[num2]))*(log_gM1x[num2+1] - np.log(geoM1x[num2])))))
#     st_geoM1y[num2+1] = np.exp(np.sqrt(np.mean((log_gM1y[num2+1] - np.log(geoM1y[num2]))*(log_gM1y[num2+1] - np.log(geoM1y[num2])))))
#     st_geoM1z[num2+1] = np.exp(np.sqrt(np.mean((log_gM1z[num2+1] - np.log(geoM1z[num2]))*(log_gM1z[num2+1] - np.log(geoM1z[num2])))))
#     st_geoM2xy[num2+1] = np.exp(np.sqrt(np.mean((log_gM2xy[num2+1] - np.log(geoM2xy[num2]))*(log_gM2xy[num2+1] - np.log(geoM2xy[num2])))))
#     st_geoM2xz[num2+1] = np.exp(np.sqrt(np.mean((log_gM2xz[num2+1] - np.log(geoM2xz[num2]))*(log_gM2xz[num2+1] - np.log(geoM2xz[num2])))))
#     st_geoM2yz[num2+1] = np.exp(np.sqrt(np.mean((log_gM2yz[num2+1] - np.log(geoM2yz[num2]))*(log_gM2yz[num2+1] - np.log(geoM2yz[num2])))))
#     st_geoM3[num2+1] = np.exp(np.sqrt(np.mean((log_gM3[num2+1] - np.log(geoM3[num2]))*(log_gM3[num2+1] - np.log(geoM3[num2])))))

#     geoD1x[num2] = stat.gmean(gD1x[num2+1])
#     geoD1y[num2] = stat.gmean(gD1y[num2+1])
#     geoD1z[num2] = stat.gmean(gD1z[num2+1])
#     geoD2xy[num2] = stat.gmean(gD2xy[num2+1])
#     geoD2xz[num2] = stat.gmean(gD2xz[num2+1])
#     geoD2yz[num2] = stat.gmean(gD2yz[num2+1])
#     geoD3[num2] = stat.gmean(gD3[num2+1])

    geoD1x[num2] = geoM1x[num2]/time2[num2]
    geoD1y[num2] = geoM1y[num2]/time2[num2]
    geoD1z[num2] = geoM1z[num2]/time2[num2]
    geoD2xy[num2] = geoM2xy[num2]/time2[num2]
    geoD2xz[num2] = geoM2xz[num2]/time2[num2]
    geoD2yz[num2] = geoM2yz[num2]/time2[num2]
    geoD3[num2] = geoM3[num2]/time2[num2]

    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM1z[num2] = np.mean(gM1z[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])
    arM2xz[num2] = np.mean(gM2xz[num2+1])
    arM2yz[num2] = np.mean(gM2yz[num2+1])
    arM3[num2] = np.mean(gM3[num2+1])
    
    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM1z[num2] = np.std(gM1z[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])
    st_arM2xz[num2] = np.std(gM2xz[num2+1])
    st_arM2yz[num2] = np.std(gM2yz[num2+1])
    st_arM3[num2] = np.std(gM3[num2+1])


#     arD1x[num2] = np.mean(gD1x[num2+1])
#     arD1y[num2] = np.mean(gD1y[num2+1])
#     arD1z[num2] = np.mean(gD1z[num2+1])
#     arD2xy[num2] = np.mean(gD2xy[num2+1])
#     arD2xz[num2] = np.mean(gD2xz[num2+1])
#     arD2yz[num2] = np.mean(gD2yz[num2+1])
#     arD3[num2] = np.mean(gD3[num2+1])

    arD1x[num2] = arM1x[num2]/time2[num2]
    arD1y[num2] = arM1y[num2]/time2[num2]
    arD1z[num2] = arM1z[num2]/time2[num2]
    arD2xy[num2] = arM2xy[num2]/time2[num2]
    arD2xz[num2] = arM2xz[num2]/time2[num2]
    arD2yz[num2] = arM2yz[num2]/time2[num2]
    arD3[num2] = arM3[num2]/time2[num2]

    st_arD1x[num2] = st_arM1x[num2]/time2[num2]
    st_arD1y[num2] = st_arM1y[num2]/time2[num2]
    st_arD1z[num2] = st_arM1z[num2]/time2[num2]
    st_arD2xy[num2] = st_arM2xy[num2]/time2[num2]
    st_arD2xz[num2] = st_arM2xz[num2]/time2[num2]
    st_arD2yz[num2] = st_arM2yz[num2]/time2[num2]
    st_arD3[num2] = st_arM3[num2]/time2[num2]

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [31]:
#np.log(gM1x[1]) - np.log(geoM1x[0])
#np.exp(np.sqrt(np.log(gM1x[2])*np.log(gM1x[2])))
#st_geoM1x
#np.exp(np.sqrt(np.mean((log_gM1x[4+1] - np.log(geoM1x[4]))*(log_gM1x[4+1] - np.log(geoM1x[4])))))

In [32]:
#np.exp((np.mean(log_gM1x[4+1] - np.log(geoM1x[4])))*(np.mean(log_gM1x[4+1] - np.log(geoM1x[4]))))

In [35]:
def plot_Mcomp(Mgeo, Mar, MFBF, t1, t2, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Mgeo = Geometric average dataset
    Mar = Arithmetic average dataset
    MFBF = Frame-by-Frame dataset
    
    t1 = time array for Mgeo and Mar
    t2 time array for MFDF
    """
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t1[1:-1], Mgeo[1:-1], linewidth=10, label='Geometric', color='blue')
    ax.plot(t1[1:-1], Mar[1:-1], linewidth=10, label='Arithmetic', color='red')
    ax.plot(t2, MFBF, linewidth=10, label='Frame-by-Frame', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.60, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return Mar, Mgeo

In [36]:
one1, bleh = plot_Mcomp(geoD2xy, arD2xy, FD2xy, time2, time, 0, 3, 'Deff (um^2/s)', Cplot, 30.1, 0.011, 6, 0.002)
plt.show()

In [37]:
arD2xy

array([        nan,  0.05862554,  0.05678791,  0.03854822,  0.03330739,
        0.02159943,  0.0195208 ,  0.01867387,  0.02261256,  0.01669942,
        0.01344005,  0.01219762,  0.01139977,  0.01469717,  0.01714821,
        0.02218891,  0.01788003,  0.0182257 ,  0.01785153,  0.        ])

In [38]:
def plot_histogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """
    total1 = total
    MSD = M

    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]

    hist = [x for x in hist if str(x) != 'nan']
    
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
#     plt.xlabel('D (um2/s)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [39]:
def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx], idx

time_a, tindex = find_nearest(time2, 1)
geoDat1 = geoD2xy[tindex]
arDat1 = arD2xy[tindex]

time_a1, tindex1 = find_nearest(time, 1)
FDat1 = FD2xy[tindex1]

print('geometric average at 1s:', geoDat1, 'um2/s with', tots, 'particles')
print('arithmetic average at 1s:', arDat1, 'um2/s with', tots, 'particles')
print('frame-by-frame average at 1s:', FDat1, 'um2/s with', fifties, 'particles')

geometric average at 1s: 0.00369377609286 um2/s with 491 particles
arithmetic average at 1s: 0.0586255420605 um2/s with 491 particles
frame-by-frame average at 1s: 0.00500746481189 um2/s with 621 particles


In [40]:
a, hist, total= plot_histogram(SD2xy, time2, tots, 'Deff (um^2/s)', Hplot, 1)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [41]:
def plot_loghistogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """

    total1 = total
    MSD = M
    
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]
    
    z1, z2 = find_nearest(hist[1:], 0)
    hist = np.delete(hist[1:], z2)
    hist = [x for x in hist if str(x) != 'nan']
    loghist = np.log(hist)

    plot, bins = np.histogram(loghist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('log(D)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [42]:
hist, MSD, total = plot_loghistogram(SD2xy, time2, tots, 'Deff (um^2/s)', Hlogplot, 1)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [43]:
def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx], idx

find_nearest(time2, 1)
SD2xy[1]

masked_array(data = [-- 0.0005868197662493528 0.00015599312821103065 0.0011516141257218622
 4.5805584971156597e-05 0.00039851464018513807 0.0007551568037520313
 0.000296807051438142 0.00027776821375195387 8.694522601609764e-05
 0.00010080859255215891 0.0001595906846624273 0.00010860421500543597
 0.00014754974530906158 8.237917055583456e-05 7.681461462095958e-05
 0.00014541911767766675 5.831678484480869e-05 0.0001691572754843763
 0.00025041957571116926],
             mask = [ True False False False False False False False False False False False
 False False False False False False False False],
       fill_value = 0.0)

In [44]:
np.savetxt('geoD2xy_{}.csv'.format(name), geoD2xy, delimiter=',')
np.savetxt('arD2xy_{}.csv'.format(name), arD2xy, delimiter=',')
np.savetxt('FD2xy_{}.csv'.format(name), FD2xy, delimiter=',')

np.savetxt('geoM2xy_{}.csv'.format(name), geoM2xy, delimiter=',')
np.savetxt('arM2xy_{}.csv'.format(name), arM2xy, delimiter=',')
np.savetxt('FM2xy_{}.csv'.format(name), FM2xy, delimiter=',')